In [ ]:
!pwd
%ls
%cd /
%ls

In [ ]:
!nvidia-smi

In [ ]:
import tensorrt as tr
import tensorflow as tf
import torch

print(f"tf cuda available：{tf.config.list_physical_devices('GPU')}")

# 输出带CPU，表示torch是CPU版本的，否则会是+cuxxx
print(f'torch的版本是：{torch.__version__}')
print(f'torch是否能使用cuda：{torch.cuda.is_available()}')

In [ ]:
pip install 'accelerate>=0.27.2' modelscope 

# 下载模型


In [ ]:
from modelscope import snapshot_download
model_dir = snapshot_download("ZhipuAI/chatglm3-6b", cache_dir='chatglm3-6b', revision = "master")


In [ ]:
!git config --global http.postBuffer 524288000 && git clone https://github.com/hiyouga/LLaMA-Factory.git


In [ ]:
!git config --global http.postBuffer 524288000 && git clone https://github.com/kangtsang/ft_data.git

In [ ]:
!cd LLaMA-Factory && pwd && git checkout -b 'master'

In [ ]:
!cd LLaMA-Factory && pwd && pip install -r ./requirements.txt

In [ ]:
!cd LLaMA-Factory && pwd && pip install -e .[metrics]

In [ ]:
## 训练问题分类 需要修改输出保存目录 output_dir

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python LLaMA-Factory/src/train.py \
    --stage sft \
    --do_train True \
    --model_name_or_path /hy-tmp/chatglm3-6b/ZhipuAI/chatglm3-6b \
    --finetuning_type lora \
    --template chatglm3 \
    --dataset_dir ft_data \
    --dataset classify \
    --cutoff_len 1024 \
    --learning_rate 5e-04 \
    --num_train_epochs 3.0 \
    --max_samples 100000 \
    --per_device_train_batch_size 35 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 2 \
    --save_steps 100 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --report_to none \
    --output_dir saves/glm3_train_classify_2024-05-06 \
    --fp16 True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.1 \
    --lora_target query_key_value \
    --plot_loss True 

In [ ]:
## 训练关键词提取 需要修改输出保存目录 output_dir

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python LLaMA-Factory/src/train.py \
    --stage sft \
    --do_train True \
    --model_name_or_path /hy-tmp/chatglm3-6b/ZhipuAI/chatglm3-6b \
    --finetuning_type lora \
    --template chatglm3 \
    --dataset_dir ft_data \
    --dataset keyword \
    --cutoff_len 1024 \
    --learning_rate 5e-04 \
    --num_train_epochs 3.0 \
    --max_samples 100000 \
    --per_device_train_batch_size 35 \
    --gradient_accumulation_steps 8 \
    --lr_scheduler_type cosine \
    --max_grad_norm 1.0 \
    --logging_steps 2 \
    --save_steps 100 \
    --warmup_steps 0 \
    --optim adamw_torch \
    --report_to none \
    --output_dir saves/glm3_train_keyword_2024-05-06 \
    --fp16 True \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.1 \
    --lora_target query_key_value \
    --plot_loss True 

In [ ]:
import torch

torch.cuda.empty_cache()

print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())


In [ ]:
# 压缩训练输出结果 保存文件

In [ ]:
!zip -q -r "saves_2024-05-06.zip" /hy-tmp/saves

In [ ]:
# 模型对话 /mnt/workspace/chatglm3-6b

from llmtuner import ChatModel
from llmtuner.extras.misc import torch_gc
chat_model = ChatModel(dict(
  model_name_or_path="/mnt/workspace/chatglm3-6b/ZhipuAI/chatglm3-6b",     # 使用 4 比特量化版 Llama-3-8b-Instruct 模型
  adapter_name_or_path="saves/glm3_train_2024-05-03",   # 加载之前保存的 LoRA 适配器
  finetuning_type="lora",                  # 和训练保持一致
  template="chatglm3",                     # 和训练保持一致
  quantization_bit=8,                      # 加载 4 比特量化模型
))
messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break

  if query.strip() == "clear":
    messages = []
    torch_gc()
    print("History has been removed.")
    continue

  messages.append({"role": "user", "content": query})     # 把提示词添加到消息中
  print("Assistant: ", end="", flush=True)
  response = ""
  for new_text in chat_model.stream_chat(messages):      # 流式输出
    print(new_text, end="", flush=True)
    response += new_text
  print()
  messages.append({"role": "assistant", "content": response}) # 把回答添加到消息中

torch_gc()